Extend example project from lecture and add the following functionality:

1. Checking for border (wall) collision and whether the snake is leaving the playing area.
2. Generate random position for food, so that it does not fall on a wall or a snake.
3. Add levels. For example, when the snake receives 3-4 foods or depending on score.
4. Increase speed when the user passes to the next level.
5. Add counter to score and level.
6. Comment your code.

In [1]:
# Import necessary libraries
import pygame  
import sys  # For system interactions
import copy  # For creating object copies
import random  # For random number generation
import time  # For time-related functions

# Initialize pygame
pygame.init()  

# Game settings
scale = 15  # Size of game objects (snake segments, food)
score = 0  # Player score
level = 0  # Game level
SPEED = 10  # Initial game speed (frames per second)

# Initial food position
food_x = 10  
food_y = 10  

# Create game window
display = pygame.display.set_mode((500, 500))
pygame.display.set_caption("Snake Game")  # Window title
clock = pygame.time.Clock()  # For controlling game speed

# Color definitions (RGB format)
background_top = (0, 0, 50)  # Top gradient color
background_bottom = (0, 0, 0)  # Bottom gradient color
snake_colour = (255, 137, 0)  # Snake body color
food_colour = (random.randint(1, 255), random.randint(1, 255), random.randint(1, 255))  # Random food color
snake_head = (255, 247, 0)  # Snake head color
font_colour = (255, 255, 255)  # Text color
defeat_colour = (255, 0, 0)  # Game over text color

class Snake:
    def __init__(self, x_start, y_start):
        # Initialize snake at starting position
        self.x = x_start  
        self.y = y_start  
        self.w = 15  # Width
        self.h = 15  # Height
        self.x_dir = 1  # Horizontal direction (1 = right, -1 = left)
        self.y_dir = 0  # Vertical direction (1 = down, -1 = up)
        self.history = [[self.x, self.y]]  # Position history for body segments
        self.length = 1  # Starting length

    def reset(self):
        """Reset snake to starting position"""
        self.x = 500 / 2 - scale  # Center X
        self.y = 500 / 2 - scale  # Center Y
        self.w = 15  
        self.h = 15  
        self.x_dir = 1  
        self.y_dir = 0  
        self.history = [[self.x, self.y]]  
        self.length = 1  

    def show(self):
        """Draw the snake on screen"""
        for i in range(self.length):
            if i == 0:  # Head
                pygame.draw.rect(display, snake_head, (self.history[i][0], self.history[i][1], self.w, self.h))
            else:  # Body segments
                pygame.draw.rect(display, snake_colour, (self.history[i][0], self.history[i][1], self.w, self.h))

    def check_eaten(self):
        """Check if snake ate the food"""
        if abs(self.history[0][0] - food_x) < scale and abs(self.history[0][1] - food_y) < scale:
            return True

    def check_level(self):
        """Check if player reached new level (every 5 segments)"""
        global level
        if self.length % 5 == 0:
            return True

    def grow(self):
        """Increase snake length"""
        self.length += 1
        self.history.append(self.history[self.length - 2])

    def death(self):
        """Check for collision with own tail"""
        i = self.length - 1
        while i > 0:
            if (abs(self.history[0][0] - self.history[i][0]) < self.w and 
                abs(self.history[0][1] - self.history[i][1]) < self.h and 
                self.length > 2):
                return True
            i -= 1

    def update(self):
        """Update snake position"""
        # Move body segments
        i = self.length - 1
        while i > 0:
            self.history[i] = copy.deepcopy(self.history[i - 1])
            i -= 1
        # Move head
        self.history[0][0] += self.x_dir * scale
        self.history[0][1] += self.y_dir * scale


class Food:
    def new_location(self):
        """Place food at random position"""
        global food_x, food_y
        food_x = random.randrange(1, int(500 / scale) - 1) * scale
        food_y = random.randrange(1, int(500 / scale) - 1) * scale

    def show(self):
        """Draw food on screen"""
        pygame.draw.rect(display, food_colour, (food_x, food_y, scale, scale))


def show_score():
    """Display current score"""
    font = pygame.font.SysFont(None, 20)
    text = font.render("Score: " + str(score), True, font_colour)
    display.blit(text, (scale, scale))


def show_level():
    """Display current level"""
    font = pygame.font.SysFont(None, 20)
    text = font.render("Level: " + str(level), True, font_colour)
    display.blit(text, (90 - scale, scale))


def gameLoop():
    """Main game loop"""
    global score, level, SPEED

    snake = Snake(500 / 2, 500 / 2)  # Create snake
    food = Food()  # Create food
    food.new_location()  # Place first food

    while True:  # Game loop
        # Event handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:  # Window closed
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:  # Key pressed
                if event.key == pygame.K_q:  # Quit key
                    pygame.quit()
                    sys.exit()
                # Direction changes (prevent 180° turns)
                if snake.y_dir == 0:  # Moving horizontally
                    if event.key == pygame.K_UP:
                        snake.x_dir = 0
                        snake.y_dir = -1
                    if event.key == pygame.K_DOWN:
                        snake.x_dir = 0
                        snake.y_dir = 1
                if snake.x_dir == 0:  # Moving vertically
                    if event.key == pygame.K_LEFT:
                        snake.x_dir = -1
                        snake.y_dir = 0
                    if event.key == pygame.K_RIGHT:
                        snake.x_dir = 1
                        snake.y_dir = 0

        # Draw gradient background
        for y in range(500):
            color = (
                background_top[0] + (background_bottom[0] - background_top[0]) * y / 500,
                background_top[1] + (background_bottom[1] - background_top[1]) * y / 500,
                background_top[2] + (background_bottom[2] - background_top[2]) * y / 500
            )
            pygame.draw.line(display, color, (0, y), (500, y))

        # Game objects
        snake.show()
        snake.update()
        food.show()
        show_score()
        show_level()

        # Food collision
        if snake.check_eaten():
            food.new_location()
            score += random.randint(1, 5)  # Random score increase
            snake.grow()

        # Level up
        if snake.check_level():
            food.new_location()
            level += 1
            SPEED += 1  # Increase game speed
            snake.grow()

        # Death condition
        if snake.death():
            score = 0  # Reset score
            level = 0  # Reset level
            font = pygame.font.SysFont(None, 100)
            text = font.render("Game Over!", True, defeat_colour)
            display.blit(text, (50, 200))
            pygame.display.update()
            time.sleep(3)  # Pause before reset
            snake.reset()

        # Screen wrapping (teleport to opposite side)
        if snake.history[0][0] > 500:  # Right edge
            snake.history[0][0] = 0
        if snake.history[0][0] < 0:  # Left edge
            snake.history[0][0] = 500
        if snake.history[0][1] > 500:  # Bottom edge
            snake.history[0][1] = 0
        if snake.history[0][1] < 0:  # Top edge
            snake.history[0][1] = 500

        pygame.display.update()  # Refresh screen
        clock.tick(SPEED)  # Control game speed


# Start the game
gameLoop()

pygame 2.6.1 (SDL 2.28.4, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\admin\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
